In [130]:
import pandas as pd
import os

In [131]:
df = pd.read_csv('./raw_data/' + os.listdir('raw_data')[1])

In [132]:
df.columns = [i.lower() for i in df.columns]

In [133]:
df.loc[0]

player                   1. Connor McDavid
team                                   EDM
age                                     22
date of birth                 Jan 13, 1997
country                             Canada
weight                     193 lbs - 88 kg
height                       6' 1 - 185 cm
pos                                      C
handed                                Left
drafted           1 - Round 1 - 2015 (EDM)
slide cand.                            NaN
waivers exempt                         NaN
arb. elig                              NaN
gp                                       0
g                                        0
a                                        0
p                                        0
p/gp                                  0.00
+/-                                      0
sh                                       0
sh%                                   0.00
toi                                    NaN
w                                        -
l          

In [134]:
df['player'] = [i.split(' ', maxsplit=1)[1] for i in df['player']]

In [135]:
df['date of birth'] = pd.to_datetime(df['date of birth'])

In [136]:
df['weight'] = [int(i.split(' ', maxsplit=1)[0]) for i in df['weight']]

In [137]:
df['height'] = [int(i.split('-')[1].split(' ')[1]) for i in df['height']]

In [138]:
df = df[df['pos'] != 'G']

df['pos'] = ['D' if i.__contains__('D') else 'F' for i in df['pos']]

In [139]:
df['drafted'] = [i.split(' ')[0] for i in df['drafted']]

In [140]:
df['slide cand.'] = [1 if i == '✓' else 0 for i in df['slide cand.']]

In [141]:
df['waivers exempt'] = [1 if i == '✓' else 0 for i in df['waivers exempt']]

In [142]:
df['arb. elig'] = [1 if i == '✓' else 0 for i in df['arb. elig']]

In [143]:
drop_cols = [
    'gp', 'g', 'a', 'p', 'p/gp', '+/-', 'sh', 'sh%', 'toi',
    'w', 'l', 'so', 'gaa', 'sv%', 'signing date', 'arb. elig.1',
    'cap hit', 'salary', 'base salary', 'minors', 's.bonus', 'p.bonus',
    'arb. req', 'exp. year'
]

df = df.drop(drop_cols, axis=1)

In [144]:
type_map = {
    'Standard (1-way)': 'one_way',
    '35+ (1-way)': 'one_way',
    'Standard (2-way)': 'two_way',
    'Entry-Level': 'elc'
}

df['type'] = df['type'].map(type_map)

In [145]:
df['clause'] = df['clause'].fillna('none')

In [146]:
df['cap hit %'] = [float(i.replace('%','')) for i in df['cap hit %']]

In [147]:
df['aav'] = [float(i.replace('$','').replace(',','')) for i in df['aav']]

In [148]:
df.head()

,player,team,age,date of birth,country,weight,height,pos,handed,drafted,...,waivers exempt,arb. elig,type,signing age,clause,length,signing,expiry,cap hit %,aav
0,Connor McDavid,EDM,22,1997-01-13,Canada,193,185,F,Left,1,...,0,0,one_way,20,none,8,RFA,UFA,16.7,12500000.0
1,Artemi Panarin,NYR,27,1991-10-30,Russia,168,180,F,Right,-,...,0,0,one_way,27,NMC,7,UFA,UFA,14.3,11642857.0
2,Auston Matthews,TOR,21,1997-09-17,United States,223,191,F,Left,1,...,0,0,one_way,21,none,5,RFA,UFA,14.6,11634000.0
3,Erik Karlsson,SJS,29,1990-05-31,Sweden,190,183,D,Right,15,...,0,0,one_way,29,NMC,8,UFA,UFA,14.5,11500000.0
4,John Tavares,TOR,28,1990-09-20,Canada,209,185,F,Left,1,...,0,0,one_way,27,NMC,7,UFA,UFA,13.8,11000000.0


In [149]:
df.loc[0]

player                 Connor McDavid
team                              EDM
age                                22
date of birth     1997-01-13 00:00:00
country                        Canada
weight                            193
height                            185
pos                                 F
handed                           Left
drafted                             1
slide cand.                         0
waivers exempt                      0
arb. elig                           0
type                          one_way
signing age                        20
clause                           none
length                              8
signing                           RFA
expiry                            UFA
cap hit %                        16.7
aav                          1.25e+07
Name: 0, dtype: object

# Combine with Career totals prior to signing

In [150]:
players = pd.read_csv('./raw_data/' + os.listdir('raw_data')[0])

In [151]:
import difflib

In [154]:
sample = 'sebastian aho'

difflib.get_close_matches(sample, players['person_fullName'])

['Sebastian Aho', 'Sebastian Aho', 'Sebastien Caron']

In [156]:
player_map = dict(zip(players['person_fullName'], players['person_id']))

In [168]:
df[df['type'] != 'elc'].player.value_counts().head(10)

Nathan Walker         2
Matt Roy              2
Peter Cehlárik        2
Colton Parayko        1
Andreas Athanasiou    1
Tyler Lewington       1
T.J. Oshie            1
Sean Monahan          1
Radek Faksa           1
Kyle Burroughs        1
Name: player, dtype: int64

In [172]:
df[df['player']=='Nathan Walker']

,player,team,age,date of birth,country,weight,height,pos,handed,drafted,...,waivers exempt,arb. elig,type,signing age,clause,length,signing,expiry,cap hit %,aav
1250,Nathan Walker,STL,25,1994-02-07,United Kingdom,186,175,F,Left,89,...,0,0,two_way,25,none,2,UFA,UFA,0.9,700000.0


In [171]:
df = df.drop(index=1245, axis=0)

In [170]:
df[df['player']=='Matt Roy']

,player,team,age,date of birth,country,weight,height,pos,handed,drafted,...,waivers exempt,arb. elig,type,signing age,clause,length,signing,expiry,cap hit %,aav
1246,Matt Roy,LAK,24,1995-03-01,United States,200,185,D,Right,194,...,1,0,two_way,24,none,2,RFA,RFA,0.9,700000.0
1251,Matt Roy,LAK,24,1995-03-01,United States,200,185,D,Right,194,...,1,0,two_way,24,none,2,RFA,RFA,0.9,700000.0


In [173]:
df = df.drop(index=1246, axis=0)

In [175]:
df[df['player']=='Peter Cehlárik']

,player,team,age,date of birth,country,weight,height,pos,handed,drafted,...,waivers exempt,arb. elig,type,signing age,clause,length,signing,expiry,cap hit %,aav
1299,Peter Cehlárik,BOS,23,1995-08-02,Slovakia,202,188,F,Left,90,...,0,0,two_way,23,none,1,RFA,RFA,0.9,700000.0
1300,Peter Cehlárik,BOS,23,1995-08-02,Slovakia,202,188,F,Left,90,...,0,0,two_way,23,none,1,RFA,RFA,0.9,700000.0


In [176]:
df = df.drop(index=1299, axis=0)

In [177]:
df[df['type'] != 'elc'].player.value_counts().head(10)

Bryan Rust            1
Andreas Athanasiou    1
Carl Dahlström        1
Tyler Lewington       1
T.J. Oshie            1
Sean Monahan          1
Radek Faksa           1
Kyle Burroughs        1
Vinnie Hinostroza     1
Nic Dowd              1
Name: player, dtype: int64

In [178]:
df = df[df['type'] != 'elc']

In [180]:
player_map = dict(zip(players['person_fullName'], players['person_id']))

df['player_id'] = [player_map[difflib.get_close_matches(i, players['person_fullName'])[0]] for i in df['']]

KeyError: ''